## Time efficiency

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load
df = pd.read_csv('../../results/times.csv')

# Fixed order
horizons = [1, 3, 7]
models   = ["LSTM", "CNNLSTM", "fine_tuned"]
model_labels = ["LSTM (C)", "CNN-LSTM (C)", "CNN-LSTM (U+FT)"]
model_colors = {"LSTM":"tab:blue", "CNNLSTM":"tab:orange", "fine_tuned":"gray"}

# Matrices: rows=horizon, cols=model
def to_matrix(metric):
    return (
        df[df["model_type"].isin(models)]
        .pivot_table(index="horizon", columns="model_type", values=metric, aggfunc="mean")
        .reindex(index=horizons, columns=models)
        .fillna(0.0)
        .to_numpy()
    )

mat_total = to_matrix("total")
mat_avg   = to_matrix("per_model")

x = np.arange(len(horizons))  # group centers = horizons
group_width = 0.8
w = group_width / len(models)
offsets = np.linspace(-group_width/2 + w/2, group_width/2 - w/2, len(models))

fig, axes = plt.subplots(1, 2, figsize=(12, 4.5), sharey=False)

def plot_panel(ax, mat, title, ylabel):
    bars = []
    for j, m in enumerate(models):
        vals = mat[:, j]
        xpos = x + offsets[j]
        b = ax.bar(xpos, vals, width=w, color=model_colors[m], label=model_labels[j])
        bars.append((b, vals, xpos))
    # annotate values
    for b, vals, xpos in bars:
        for xi, yi in zip(xpos, vals):
            ax.text(xi, yi, f"{yi:.2f}", ha="center", va="bottom", fontsize=9)
    ax.set_xticks(x)
    ax.set_xticklabels([f"H={h}" for h in horizons])
    ax.set_xlabel("Horizon")
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.legend(title="Model")
    ax.grid(axis='y', linestyle=':', linewidth=0.5, alpha=0.6)

plot_panel(axes[0], mat_total, "Total Duration (seconds)", "Seconds")
plot_panel(axes[1], mat_avg,   "Average Duration Per Model (seconds)", "Seconds")

plt.tight_layout()
plt.show()


## Performance boxplots

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# --- load ---
metrics = pd.read_csv('../../logs/metrics.csv')
metrics = metrics[metrics['type'] == "test"]
details = pd.read_csv('../../logs/details.csv')
details = details[details['type'].isin(['LSTM', 'CNNLSTM', 'fine_tuned'])]
merged = metrics.merge(details, on="model", suffixes=("_metric", "_detail"))

# --- collapse over consumers (median across experiments per consumer) ---
collapsed = (merged
    .groupby(['type_detail', 'horizon_detail', 'dataset'], as_index=False)
    .agg({'MSE':'median','MAPE':'median'}))

# config
models     = ["LSTM", "CNNLSTM", "fine_tuned"]
model_lbl  = {"LSTM":"LSTM (C)", "CNNLSTM":"CNN-LSTM (C)", "fine_tuned":"CNN-LSTM (U+FT)"}
model_col  = {"LSTM":"tab:blue", "CNNLSTM":"tab:gray", "fine_tuned":"tab:orange"}
horizons   = sorted(collapsed['horizon_detail'].unique())

def grouped_boxplot_by_horizon(ax, metric, show_outliers=False):
    base_x = np.arange(len(horizons))
    m = len(models)
    box_w, gap = 0.23, 0.02
    offsets = [(j - (m - 1) / 2) * (box_w + gap) for j in range(m)]

    handles, labels = [], []
    for j, mod in enumerate(models):
        # one array per horizon (so positions align to horizons)
        data_by_h = [
            collapsed[(collapsed['type_detail']==mod) & (collapsed['horizon_detail']==h)][metric].values
            for h in horizons
        ]
        pos = base_x + offsets[j]
        bp = ax.boxplot(
            data_by_h,
            positions=pos,
            widths=box_w,
            patch_artist=True,
            showfliers=False,
            showmeans=True,
            meanline=False,
            meanprops=dict(marker='x', markeredgecolor='black'),
            medianprops=dict(color='black'),
            whis=1.5,
            manage_ticks=False,
        )
        for b in bp["boxes"]:
            b.set_facecolor(model_col[mod]); b.set_alpha(0.65)
        handles.append(bp["boxes"][0]); labels.append(model_lbl[mod])

    ax.set_xticks(base_x)
    ax.set_xticklabels([f"H={h}" for h in horizons])
    ax.set_xlabel("Horizon")
    ax.set_ylabel(metric)
    ax.set_title(metric)
    ax.grid(axis="y", linestyle=":", linewidth=0.6, alpha=0.6)
    ax.legend(handles, labels, title="Model")

fig, axes = plt.subplots(1, 2, figsize=(12, 4.5), dpi=200)
grouped_boxplot_by_horizon(axes[0], "MSE")
grouped_boxplot_by_horizon(axes[1], "MAPE")
plt.tight_layout()
plt.show()


## Qualitative analysis

In [ ]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import json

data = pd.read_csv("../../logs/data.csv")
data["y_true"] = data["y_true"].apply(lambda s: np.asarray(json.loads(s), dtype=np.float64))
data["y_pred"] = data["y_pred"].apply(lambda s: np.asarray(json.loads(s), dtype=np.float64))

metrics = pd.read_csv('../../logs/metrics.csv')
metrics = metrics[metrics['type'] == "test"]
metrics = metrics.drop(columns=['experiment'])

details = pd.read_csv('../../logs/details.csv')
details = details[details['type'].isin(['LSTM', 'CNNLSTM', 'fine_tuned'])]

merged = metrics.merge(
        details,
        left_on="model",
        right_on="model",
        suffixes=("_metric", "_detail"),
)
consumers = details['dataset'].unique()
merged = merged[merged['experiment'] == 7]


for consumer in consumers:
        print(f"Consumer {consumer}")
        for h in [1, 3, 7]:
                subset = merged[(merged['dataset'] == consumer) & (merged['horizon_detail'] == h)].copy()
                subset_lstm = subset[subset['type_detail'] == "LSTM"].copy()
                subset_cnnlstm = subset[subset['type_detail'] == "CNNLSTM"].copy()
                subset_ft = subset[subset['type_detail'] == "fine_tuned"].iloc[0]

                best_lstm = subset_lstm.sort_values("MAPE", kind="stable").iloc[0]
                best_cnnlstm = subset_cnnlstm.sort_values("MAPE", kind="stable").iloc[0]


                lstm_pred = data.loc[data['model'] == best_lstm['model'], 'y_pred'].iloc[0].ravel()
                cnnlstm_pred = data.loc[data['model'] == best_cnnlstm['model'], 'y_pred'].iloc[0].ravel()

                ft_row = data.loc[data['model'] == subset_ft['model']].iloc[0]
                y_true = np.asarray(ft_row['y_true']).ravel()
                ft_pred = np.asarray(ft_row['y_pred']).ravel()


                plt.figure(figsize=(15, 7))
                plt.title(f"Forecast of the consumer {consumer} at the horizon {h}")
                plt.plot(y_true, color='blue', linewidth=1.0, label='True values')
                plt.plot(lstm_pred, color='green', linewidth=1.0, label='LSTM (C)')
                plt.plot(cnnlstm_pred, color='yellow', linewidth=1.0, label='CNN-LSTM (C)')
                plt.plot(ft_pred, color='red', linewidth=1.0, label='CNN-LSTM (U+FT)')
                plt.legend()
                plt.ylabel("Load consumption (kWh)")
                plt.xlabel("Date")
                plt.show()